<a href="https://colab.research.google.com/github/1smiley43/Project-2/blob/main/Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
## Pandas
import pandas as pd
## Numpy
import numpy as np
## MatplotLib
import matplotlib.pyplot as plt

## Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

## Models
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor

## Regression Metrics
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

## Set global scikit-learn configuration 
from sklearn import set_config
## Display estimators as a diagram
set_config(display='diagram') # 'text' or 'diagram'}

#Dataset 1

In [6]:
## Loading data
df = pd.read_csv('/content/drive/MyDrive/Coding Dojo files/adult.csv')
df.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


Source of data:

https://www.kaggle.com/datasets/wenruliu/adult-income-dataset

Description of data:

This data houses information on individuals and their income bracket, at either below $50K or equal/above $50K.

The Target:

The target is to predict if a person's 
income will earn below or above $50K.

The Rows:

Each row represents one person in this dataset.

Regression or Classification:

Since there are only two options for the result, this is a classification problem.

In [7]:
df.shape

(48842, 15)

Features and Rows:

This dataset has 48842 rows and 15 features.



Possible Challenges:

There are values of "?" that will need to be addressed. There are many values that will need to be OneHotEncoded.

#Dataset 2

In [ ]:
## Loading data
df = pd.read_csv('/content/drive/MyDrive/Coding Dojo files/ufc_fighters.csv')
df.head()

,First Name,Last Name,Nickname,Height,Weight,Reach,Stance,Wins,Losses,Draws
0,Tom,Aaron,NaN,--,155 lbs.,--,NaN,5,3,0
1,Danny,Abbadi,The Assassin,"5' 11""",155 lbs.,--,Orthodox,4,6,0
2,Nariman,Abbasov,Bayraktar,"5' 8""",155 lbs.,"66.0""",Orthodox,28,4,0
3,David,Abbott,Tank,"6' 0""",265 lbs.,--,Switch,10,15,0
4,Hamdy,Abdelwahab,The Hammer,"6' 2""",264 lbs.,"72.0""",Southpaw,5,0,0


Source of data: 

https://www.kaggle.com/datasets/fatismajli/ufc-data

Description of data: 

This data contains stats for UFC fighters including their weight, height, reach, and stance, along with their win-lose-draw ratios.

The Target:

The target of this dataset is to predict if a fighter will win.

The Rows:

Each row represents one fighter and their information.

Regression or Classification:

This is a Classification problem.

In [ ]:
df.shape

(3951, 10)

Features and Rows:

There are 3951 rows and 10 features.

Possible challenges:

There are some missing values in the data that might skew the data depending on how I decide how to impute them. I also notice that the height, weight, and reach are all objects that will need to be manipulated into numeric formats.